# مروری کلی بر کرونا
> مروری بر کل موارد کرونا ، مرگ و میر و موارد جدید براساس کشور.

- comments: true
- author: پراتاب واردهان و سولماز عباسپور
- categories: [مرور]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
!pip install jdatetime
!pip install persiantools
import numpy as np
import pandas as pd
from jinja2 import Template
import jdatetime as jd
import os
from persiantools import digits
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
dirpath = os.path.dirname(os.getcwd())
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': dirpath + '/_templates/overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
0,US,699706,36773,555313,22019,144393,14754,5.3,North America
1,Spain,190839,20002,166831,17209,24008,2793,10.5,Europe
2,Italy,172434,22745,156363,19899,16071,2846,13.2,Europe
3,France,149130,18703,121712,14412,27418,4291,12.5,Europe
4,Germany,141397,4352,127854,3022,13543,1330,3.1,Europe
5,United Kingdom,109769,14607,85206,10629,24563,3978,13.3,Europe
6,China,83760,4636,83134,3343,626,1293,5.5,Asia
7,Iran,79494,4958,71686,4474,7808,484,6.2,Asia
8,Turkey,78546,1769,56956,1198,21590,571,2.3,Asia
9,Belgium,36138,5163,29647,3600,6491,1563,14.3,Europe


In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
s_iran = df_table[df_table['Country/Region'].eq('Iran')][metrics].sum().add_prefix('Iran ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu, **s_iran}
summary

{'updated': Timestamp('2020-04-17 00:00:00'),
 'since': Timestamp('2020-04-12 00:00:00'),
 'Cases': 2239710,
 'Deaths': 153809,
 'Cases (+)': 405272,
 'Deaths (+)': 39726,
 'China Cases': 83760,
 'China Deaths': 4636,
 'China Cases (+)': 626,
 'China Deaths (+)': 1293,
 'US Cases': 699706,
 'US Deaths': 36773,
 'US Cases (+)': 144393,
 'US Deaths (+)': 14754,
 'EU Cases': 1030437,
 'EU Deaths': 96264,
 'EU Cases (+)': 160169,
 'EU Deaths (+)': 19464,
 'Iran Cases': 79494,
 'Iran Deaths': 4958,
 'Iran Cases (+)': 7808,
 'Iran Deaths (+)': 484}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,21,40,37,34,52,58,49,70,56,66
Albania,0,0,0,0,0,0,0,0,0,0,...,17,9,7,17,13,21,8,19,24,21
Algeria,0,0,0,0,0,0,0,0,0,0,...,104,94,95,64,89,69,87,90,108,150
Andorra,0,0,0,0,0,0,0,0,0,0,...,19,19,18,0,37,8,13,14,0,23
Angola,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0


In [9]:
#hide_input
template = Template(get_template(paths['overview']))
days = digits.en_to_fa(str((summary['updated'] - summary['since']).days))
g_date_updated = pd.to_datetime(summary['updated'])
jdate_updated = jd.date.fromgregorian(year=g_date_updated.year,month=g_date_updated.month,day=g_date_updated.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_updated = digits.en_to_fa(jdate_updated)
g_date_since = pd.to_datetime(summary['since'])
jdate_since = jd.date.fromgregorian(year=g_date_since.year,month=g_date_since.month,day=g_date_since.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_since = digits.en_to_fa(jdate_since)
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, days=days, digits=digits, 
    jdate_updated=jdate_updated, jdate_since=jdate_since, enumerate=enumerate)
HTML(f'<div>{html}</div>')


تحلیل اصلی از [پراتاپ واردهان](https://twitter.com/PratapVardhan)[^۱]

[^۱]: منبع: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). لینک به صفحه [اصلی](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-trajectories.ipynb).